# This notebook will implement Deep learning techniques for a particle-in-halo classification framework. Refer to the original publicly notebook in [ChJazhiel notebook](https://github.com/ChJazhiel/ML_ICF/blob/master/RF_Particles_z23.ipynb)

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from matplotlib.ticker import AutoMinorLocator

%matplotlib inline

# workdirectory = '/home/jazhiel/ML_Notebooks/Cosmology_ML/'
workdirectory = ''

## Import our dataset 

In [5]:
data_dict = np.load(workdirectory + 'OUTFILE1M.npz') #np.load('/path/to/nbody/outfile.npz')
test_flags  = data_dict['test_flags'] ## not important
test_hosts  = data_dict['test_hosts'] ### somewhat relevant
test_mass   = data_dict['test_mass'] ## important
test_labels = data_dict['test_labels'] ## important
test_input  = data_dict['test_input'] ## very important
#test_snid   = dict_data['test_snid']
#test_labels = dict_data['test_labels']
print(test_mass) ## Here I want to check how is the halo mass matrix composed of, the -1 means the halo 
#is not in our range of 10^12-10^13 M_sun
print(np.sum(test_labels)) ## Here I want to check how many label "1" do we have

[-1.00000000e+00  2.72242898e+13  1.13894322e+13 ... -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00]
289964


## Data preprocessing

In [6]:
### here we create or 10 vector dataset, I wonder if by adding some other information the classification will be better
### adding mass is not helpful, the classifier is perfect in that regard
dr1 = pd.DataFrame(test_input[0], columns = ['dr1'])
dr2 = pd.DataFrame(test_input[1], columns = ['dr2'])
dr3 = pd.DataFrame(test_input[2], columns = ['dr3'])
dr4 = pd.DataFrame(test_input[3], columns = ['dr4'])
dr5 = pd.DataFrame(test_input[4], columns = ['dr5'])
dr6 = pd.DataFrame(test_input[5], columns = ['dr6'])
dr7 = pd.DataFrame(test_input[6], columns = ['dr7'])
dr8 = pd.DataFrame(test_input[7], columns = ['dr8'])
dr9 = pd.DataFrame(test_input[8], columns = ['dr9'])
dr10 = pd.DataFrame(test_input[9], columns = ['dr10'])
#mass = pd.DataFrame(test_mass, columns = ['Halo_Mass'])
lbl = pd.DataFrame(test_labels, columns =['labels'])

## Select all features and create dataframe

In [7]:
df = pd.concat([dr1, dr2, dr3, dr4, dr5, dr6, dr7, dr8, dr9, dr10, lbl], axis=1, ignore_index=False, sort=False)
df

,dr1,dr2,dr3,dr4,dr5,dr6,dr7,dr8,dr9,dr10,labels
0,-0.209889,-0.187730,-0.169542,-0.155964,-0.100744,-0.049741,-0.011099,-0.010984,-0.002834,0.024089,0
1,0.364046,0.430008,0.453121,0.450021,0.374578,0.309706,0.240559,0.190503,0.174401,0.176335,1
2,0.311775,0.254511,0.227109,0.215286,0.148460,0.112097,0.090454,0.058546,0.035873,0.043613,1
3,0.033877,0.051836,0.065151,0.073930,0.100935,0.100271,0.094132,0.080007,0.059981,0.032160,0
4,-0.355428,-0.330448,-0.306125,-0.291701,-0.253399,-0.215256,-0.163749,-0.124680,-0.080408,-0.051999,0
...,...,...,...,...,...,...,...,...,...,...,...
999995,0.022223,0.034349,0.046298,0.047812,0.027945,0.018082,0.012005,0.015623,0.028762,0.036779,0
999996,-0.178963,-0.188312,-0.189045,-0.187326,-0.182576,-0.170714,-0.155041,-0.143078,-0.136640,-0.120222,0
999997,-0.164632,-0.113120,-0.076089,-0.061724,0.009549,0.039809,0.057906,0.041327,0.010610,-0.020231,0
999998,-0.039698,0.063765,0.118363,0.140928,0.152761,0.125845,0.114529,0.113929,0.107155,0.098008,0


## Here we select a dataframe consisting of evenly separated labels

(I'm not sure if selecting all particles will impact in a different result)


In [8]:
## Sorting out the labels consisting in label '0' and label '1'
## Then we sample them in order to not selecting them in a specific range or shape

df_0 = df.sort_values('labels').head(710036).sample(289000)
df_1 = df.sort_values('labels').tail(289964).sample(289000) 
df_1.labels.sum()
df_r = pd.concat([df_0, df_1])
#df_scaled = scaler.fit_transform(df_r.drop(['labels'], axis = 1))  
#df_copy1 = pd.DataFrame(df_scaled, columns=['dr1', 'dr2', 'dr3', 'dr4', 'dr5', 'dr6', 'dr7', 'dr8', 'dr9', 'dr10'])

shuffle_df = df_r.sample(frac = 1.0)
#shuffle_df.insert(10, "Model", ['LC','SF']*14300, True)
shuffle_df

,dr1,dr2,dr3,dr4,dr5,dr6,dr7,dr8,dr9,dr10,labels
114074,0.026622,-0.006134,-0.006339,-0.002562,0.027857,0.060640,0.087173,0.114187,0.121830,0.102355,0
247021,0.076285,0.081491,0.101477,0.117110,0.097762,0.095314,0.085185,0.068890,0.058849,0.062756,0
925244,0.203805,0.256781,0.253981,0.238800,0.228309,0.225982,0.211441,0.174322,0.150403,0.145066,1
567886,0.094756,0.050047,0.024114,0.012384,0.005600,-0.003903,0.000206,0.007552,0.010601,0.016956,0
967527,-0.012243,0.034301,0.040747,0.041883,0.079139,0.103540,0.121837,0.147844,0.150164,0.116473,0
...,...,...,...,...,...,...,...,...,...,...,...
219424,0.156575,0.139391,0.129394,0.123158,0.149332,0.168104,0.169997,0.149111,0.146129,0.139427,1
518270,0.348880,0.323346,0.287654,0.257516,0.229198,0.180555,0.128919,0.095656,0.059576,0.037967,1
376075,-0.310457,-0.249178,-0.207620,-0.188395,-0.136709,-0.121262,-0.108247,-0.107783,-0.110706,-0.123596,0
489841,-0.001529,0.046879,0.073977,0.082090,0.084862,0.070727,0.042188,0.021427,0.032157,0.049011,0


## define a size for our traning dataset 

I think that for 500k + particles we can divide into train, test and validation datasets

In [9]:
# Define a size for your train set 
train_size = int(0.8 * len(df_r))

# Split your dataset 
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

train_set

,dr1,dr2,dr3,dr4,dr5,dr6,dr7,dr8,dr9,dr10,labels
114074,0.026622,-0.006134,-0.006339,-0.002562,0.027857,0.060640,0.087173,0.114187,0.121830,0.102355,0
247021,0.076285,0.081491,0.101477,0.117110,0.097762,0.095314,0.085185,0.068890,0.058849,0.062756,0
925244,0.203805,0.256781,0.253981,0.238800,0.228309,0.225982,0.211441,0.174322,0.150403,0.145066,1
567886,0.094756,0.050047,0.024114,0.012384,0.005600,-0.003903,0.000206,0.007552,0.010601,0.016956,0
967527,-0.012243,0.034301,0.040747,0.041883,0.079139,0.103540,0.121837,0.147844,0.150164,0.116473,0
...,...,...,...,...,...,...,...,...,...,...,...
729558,0.212113,0.183576,0.167638,0.161483,0.139855,0.120183,0.095670,0.093532,0.110854,0.108310,0
388827,-0.088783,-0.030785,0.013404,0.033890,0.080209,0.071070,0.062266,0.058827,0.036604,0.027457,1
377480,0.283100,0.249064,0.261652,0.262266,0.206714,0.177706,0.136981,0.129759,0.123402,0.094036,1
345680,0.333194,0.267164,0.215277,0.184976,0.103820,0.063887,0.023875,-0.015216,-0.037427,-0.046541,1


## Select data, X for attributes, y for labels

In [10]:
X = shuffle_df.drop(['labels'], axis = 1)
ylabels = shuffle_df.labels
#ymodel = shuffle_df.Model
#ymodel = copy1.Model
X_train = train_set.drop(['labels'], axis = 1)
X_test = test_set.drop(['labels'], axis= 1)
y_trainl = train_set.labels
y_testl = test_set.labels
#y_trainM = train_set.Model
#y_testM = test_set.Model
print(X_train.shape)

(462400, 10)


## For the Confusion matrix 


In [11]:
#### THIS CELL IS FOR CALCULATE THE CONFUSION MATRIX ####


import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
    


np.set_printoptions(precision=2)

## Random forest with sklearn 

In [13]:
rf=RandomForestClassifier(n_estimators=100, max_depth=8, n_jobs=-1,
                          criterion = 'entropy', class_weight='balanced')

## n_jobs = -1 tells my computer to use all its cores, I have only 2, so it runs on 2 cores
#Train the model using the training sets y_pred=clf.predict(X_test)
rf.fit(X_train,y_trainl)

ypred = rf.predict(X_test)
#####

print('Traning and Testing on raw data, all features \n');
#### Model accuracy
print("Accuracy:", metrics.accuracy_score(y_testl, ypred))

for i, score_forest in enumerate(cross_val_score(rf, X, ylabels, cv = 3)):
    print('Random Forest accuracy for the %d score: %0.2f' % (i, score_forest))
score_forest=cross_val_score(rf, X, ylabels, cv=3)
#score_tree
cv_scores = []
print("Random Forest Accuracy: %0.2f (+/- %0.2f)" % (score_forest.mean(), score_forest.std() * 2 ))
cv_score = score_forest.mean()
cv_scores.append(cv_score)

### We perform a cross validation score to see how accurate our decision tree is by 
### splitting the dataset into 10 folds for training and testing

Traning and Testing on raw data, all features 

Accuracy: 0.7857871972318339
Random Forest accuracy for the 0 score: 0.78
Random Forest accuracy for the 1 score: 0.78
Random Forest accuracy for the 2 score: 0.79
Random Forest Accuracy: 0.78 (+/- 0.00)


## Perceptron with Keras

In [ ]:
#para scikit-learn: (samples,features)
#test_input_T = test_input.T
#X_train, X_test, y_train, y_test = train_test_split(test_input_T,test_labels,test_size=0.25,random_state=None)

#ANN
test1_model = models.Sequential()
test1_model.add(layers.Dense(100,activation='relu',input_shape=(10,)))
test1_model.add(layers.Dense(100,activation='relu'))
#test1_model.add(layers.Dense(32,activation='relu'))
#test1_model.add(layers.Dense(10,activation='relu'))
test1_model.add(layers.Dense(1, activation='sigmoid'))

test1_model.compile(optimizer=optimizers.Adam(lr=0.0001),#RMSprop(lr=0.001),
                    loss=losses.binary_crossentropy,
                    metrics=['accuracy'])

test1_model.summary()

test1_model_history = test1_model.fit(X_train,
                                      y_trainl,
                                      epochs=50,
                                      batch_size=32,
                                      validation_data=(X_test,y_testl),
                                      verbose=1)

#plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(11,4.5))    

results_dict = test1_model_history.history
epochs = range(1,len(results_dict['accuracy'])+1)

#accuracy
acc_values = results_dict['accuracy']
val_acc_values = results_dict['val_accuracy']
    
axs[0].plot(epochs,acc_values,color='black',label='train',linewidth=1.5,linestyle='-')
axs[0].plot(epochs,val_acc_values,color='red',label='test',linewidth=1.5,linestyle='-')
axs[0].set_xlabel('epochs')
minorLocatorX = AutoMinorLocator()
axs[0].xaxis.set_minor_locator(minorLocatorX)
axs[0].set_ylabel('accuracy')
axs[0].set_ylim([0.5,1.02])   
minorLocatorY = AutoMinorLocator()
axs[0].yaxis.set_minor_locator(minorLocatorY)
axs[0].tick_params(which='major', length=6)
axs[0].tick_params(which='minor', length=3, color='black')       
axs[0].legend(loc='lower right')
    
#loss
loss_values = results_dict['loss']
val_loss_values = results_dict['val_loss']
       
axs[1].plot(epochs,loss_values,color='black',label='train',linewidth=1.5,linestyle='-')
axs[1].plot(epochs,val_loss_values,color='red',label='test',linewidth=1.5,linestyle='-')
axs[1].set_xlabel('epochs')
minorLocatorX = AutoMinorLocator()
axs[1].xaxis.set_minor_locator(minorLocatorX)   
axs[1].set_ylabel('loss')
minorLocatorY = AutoMinorLocator()
axs[1].yaxis.set_minor_locator(minorLocatorY)
axs[1].tick_params(which='major', length=6)
axs[1].tick_params(which='minor', length=3, color='black')    
axs[1].legend(loc='upper right')

axs[0].set_title('Training and test accuracy after {} epochs: \n {:.3f} (train), {:.3f} (validation)'
                 .format(len(epochs),acc_values[-1],val_acc_values[-1]));

axs[1].set_title('Training and test loss after {} epochs: \n {:.3f} (train), {:.3f} (validation)'
                 .format(len(epochs),loss_values[-1],val_loss_values[-1]));

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_13 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 101       
Total params: 11,301
Trainable params: 11,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


2022-05-09 22:29:35.321030: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


14450/14450 [==============================] - 9s 629us/step - loss: 0.4676 - accuracy: 0.7793 - val_loss: 0.4592 - val_accuracy: 0.7828
Epoch 2/50
14450/14450 [==============================] - 9s 613us/step - loss: 0.4601 - accuracy: 0.7815 - val_loss: 0.4579 - val_accuracy: 0.7838
Epoch 3/50
14450/14450 [==============================] - 9s 611us/step - loss: 0.4594 - accuracy: 0.7820 - val_loss: 0.4580 - val_accuracy: 0.7830
Epoch 4/50
14450/14450 [==============================] - 9s 606us/step - loss: 0.4591 - accuracy: 0.7818 - val_loss: 0.4575 - val_accuracy: 0.7835
Epoch 5/50
14450/14450 [==============================] - 9s 606us/step - loss: 0.4588 - accuracy: 0.7820 - val_loss: 0.4572 - val_accuracy: 0.7834
Epoch 6/50
14450/14450 [==============================] - 9s 607us/step - loss: 0.4587 - accuracy: 0.7822 - val_loss: 0.4569 - val_accuracy: 0.7837
Epoch 7/50
14450/14450 [==============================] - 9s 610us/step - loss: 0.4585 - accuracy: 0.7823 - val_loss: 0.456